In [49]:
import pandas as pd
import numpy as np 

#CSVs must have customer id as a number not scientific notation

#import two csv's to DataFrames
us_raw = pd.read_csv('us sales 4.30.21.csv').fillna(0)
can_raw = pd.read_csv('canada sales 4.30.21.csv').fillna(0)

raw = us_raw.append(can_raw)



#Group by correct columns
grouped_elements = ['total_sales','orders']
raw_grouped = raw.groupby(['month','customer_id','customer_name'])[grouped_elements].sum().astype(int).reset_index()


#Create crosstab() table 
raw_grouped_cross = pd.crosstab(raw_grouped['customer_id'],raw_grouped['month']).astype(int).reset_index()
raw_grouped_refresh = pd.crosstab(raw_grouped['customer_id'],raw_grouped['month']).astype(int).reset_index()
raw_grouped = pd.crosstab(raw_grouped['customer_id'],raw_grouped['month']).astype(int).reset_index()

# for loop to add columns summing previous 12 months to get customers

num = 12
start_col = 1
end_col = 13



for x in range(len(raw_grouped.columns)-12) : 
    
    test_series = raw_grouped.iloc[:,start_col:end_col].sum(axis=1) 
    
    new_col = test_series >0
    date = raw_grouped.columns[num]
    start_col = start_col+1
    end_col = end_col+1
    num = num+1
       
    raw_grouped['active' + str(date)] = new_col
        
        

    
    
active_customers = raw_grouped



#append a sum to the bottom of active customer df
cust_sum = active_customers[:].sum()
full = active_customers.append(cust_sum, ignore_index=True)
full_refresh = active_customers.append(cust_sum, ignore_index=True)



#slice to only show actives customers for each month
sliced = full.iloc[raw_grouped['customer_id'].count():,len(raw_grouped_refresh.columns):]
monthly_customers = full.iloc[-1:,1:-len(raw_grouped_refresh.columns)+12]






#download to csv
sliced.to_csv('/Users/evandiazdearce/Documents/output/active_customersTotal.csv', index = False, header=True)
monthly_customers.to_csv('/Users/evandiazdearce/Documents/output/monthly_customers.csv', index = False, header=True)



############## starting on Churned users ##################

cust_data = full
filt_cust_data = cust_data.iloc[:,len(raw_grouped_refresh.columns):]


num_2 = 1
start_col_2 = 0
end_col_2 = start_col_2 +1 




for x in range(len(filt_cust_data.columns)-1) : 
    
    new_col_2 = filt_cust_data.iloc[:,start_col_2] - filt_cust_data.iloc[:,end_col_2] > 0

    
    date_2 = cust_data.columns[num_2+12] 
    
    num_2 = num_2+1
    
    start_col_2 = start_col_2 +1
    end_col_2 = end_col_2 +1 
    
    filt_cust_data['is_churned' + str(date_2)] = new_col_2





cust_data_sum = -filt_cust_data[:].sum()

display(cust_data_sum)

full_churn = filt_cust_data.append(cust_data_sum,ignore_index=True)
full_churn_sliced = full_churn.iloc[-1:,-len(sliced.columns)+1:]



full_churn_sliced.to_csv('/Users/evandiazdearce/Documents/output/churned.csv', index = False, header=True)


########### starting retained Users #################

retained_cust = active_customers



num_3 = 1
rg_len = len(raw_grouped.columns)
rgc_len = len(raw_grouped_cross.columns)
rg_rgc_diff = rg_len-rgc_len
rg_start_col = rgc_len - rg_rgc_diff



for x in range(rg_rgc_diff - 1) : 
    
    new_col_3 = retained_cust.apply(lambda x: 1 if x[raw_grouped_cross.columns[rg_start_col + num_3]] == 1 and x[raw_grouped.columns[len(raw_grouped_cross.columns)-1+num_3]] == True else 0, axis=1)
    

    #print(retained_cust[raw_grouped.columns[len(raw_grouped_cross.columns)-1+num_3]])    
    date_3 = retained_cust.columns[rg_start_col + num_3]
    num_3 = num_3+1
   

    retained_cust['is_retained' + str(date_3)] = new_col_3
    

retained_sum = retained_cust[:].sum() 
full_retained = retained_cust.append(retained_sum, ignore_index = True)
sliced_retained = full_retained.iloc[-1:,-rg_rgc_diff + 1 :]


sliced_retained.to_csv('/Users/evandiazdearce/Documents/output/retained_cust.csv', index = False, header=True)




-29


month
active2019-12       -22962
active2020-01       -24294
active2020-02       -25548
active2020-03       -26478
active2020-04       -27608
active2020-05       -28946
active2020-06       -30428
active2020-07       -31930
active2020-08       -34314
active2020-09       -35786
active2020-10       -36560
active2020-11       -37528
active2020-12       -38506
active2021-01       -39126
active2021-02       -40124
active2021-03       -40726
active2021-04       -40758
is_churned2020-01     -628
is_churned2020-02     -521
is_churned2020-03     -646
is_churned2020-04     -612
is_churned2020-05     -508
is_churned2020-06     -556
is_churned2020-07     -489
is_churned2020-08     -486
is_churned2020-09     -653
is_churned2020-10    -1127
is_churned2020-11     -955
is_churned2020-12    -1399
is_churned2021-01    -1014
is_churned2021-02     -947
is_churned2021-03     -885
is_churned2021-04     -941
dtype: int64